In [ ]:
%ntbl pull datasets "hh_ru/hh_ru_dataset.csv"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('../datasets/hh_ru/hh_ru_dataset.csv', sep=',')
df.info()

In [ ]:
# финальный статус - invitation или discrard
df_cleared = df.copy()
len(df_cleared)
#len(df_cleared[df_cleared.compensation_to.isna()])

In [ ]:
df_cleared = df_cleared[-((df.final_state == 'response') | (df.final_state.isna()))]
len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
# трудоспособный возраст
df_cleared = df_cleared[(df_cleared['year_of_birth'] >= 1953) & (df_cleared['year_of_birth'] <= 2007)]
len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
# избавляемся от дубликатов уникального показателя
df_cleared = df_cleared.drop_duplicates(['topic_id'])
len(df_cleared)

In [ ]:
# проверка на логику между compensation_from и compensation_to
df_cleared = df_cleared[(df_cleared.compensation_to.isna()) | (df_cleared.compensation_to == 0.0) | (df_cleared['compensation_from'] <= df_cleared['compensation_to'])]
len(df_cleared)

In [ ]:
# избавляемся от незаполненных полей
# df_cleared = df_cleared.dropna(axis=0, subset=['education_level', 'initial_state', 'work_schedule', 'compensation_from', 'compensation_to'])
df_cleared = df_cleared.dropna(axis=0, subset=['education_level', 'initial_state'])
len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
# стаж работы с 16 лет; работал не больше, чем до пенсии
df_cleared = df_cleared[(2023 - df_cleared['work_experience_months'] / 12 - df_cleared['year_of_birth'] >= 16) & (df_cleared['work_experience_months'] <= 600)]
len(df_cleared)
# print(500000 - len(df_cleared))

In [ ]:
# ожидаемые зарплаты без выбросов
quantile_param = 0.03
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles[prof] = (df_prof['expected_salary'].quantile(quantile_param),
                       df_prof['expected_salary'].quantile(1 - quantile_param))

df_cleared.loc[:,'exp_salary_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'exp_salary_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]

# проверка по ожидаемой зарплате
df_cleared = df_cleared[(df_cleared['expected_salary'] >= df_cleared['exp_salary_min_limit']) & 
                            (df_cleared['expected_salary'] <= df_cleared['exp_salary_max_limit'])]
len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
# compensation_to без выбросов
quantiles_to = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles_to[prof] = (df_prof['compensation_to'].quantile(quantile_param),
                       df_prof['compensation_to'].quantile(1 - quantile_param))
    
df_cleared.loc[:,'compensation_to_min_limit'] = [quantiles_to[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'compensation_to_max_limit'] = [quantiles_to[i][1] for i in df_cleared['profession']]

# проверка по compensation_to
df_cleared = df_cleared[(df_cleared['compensation_to'] >= df_cleared['compensation_to_min_limit']) & 
                            (df_cleared['compensation_to'] <= df_cleared['compensation_to_max_limit'])]

len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
# замена final_state, gender и education_level
fs_dict = {'discard': 0, 'invitation': 1}
gender_dict = {'female': 0, 'male': 1}
education_dict = {'secondary': 0, 'special_secondary': 1, 'unfinished_higher': 2,
                'bachelor': 3, 'master': 4, 'higher': 5, 'candidate': 6, 'doctor': 7}
df_cleared = df_cleared.replace({'final_state': fs_dict, 'gender': gender_dict, 'education_level': education_dict})

In [ ]:
df_cleared.sort_values('expected_salary')['expected_salary']
#df_cleared.info()

In [ ]:
df_cleared.to_excel('clear_new1.xlsx')

In [ ]:
# графики по профессиям
columns = ['final_state','gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Аналитик'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Менеджер по персоналу'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Курьер'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Дизайнер, художник'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Машинист'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Тестировщик'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Учитель, преподаватель, педагог'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Юрист'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Психолог'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");


columns = ['final_state', 'gender', 'education_level', 'work_experience_months', 'expected_salary']
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(df_cleared[columns][df_cleared['profession'] == 'Врач'].corr( ), square=True,
              annot=True, fmt=".10f", linewidths=0.1, cmap="RdBu");